In [4]:
# Установка PyTorch
!pip install torch torchvision torchaudio torchtext


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [5]:
# Клонируем репозиторий StyleGAN2-ADA
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git


Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 131 (delta 0), reused 1 (delta 0), pack-reused 128
Receiving objects: 100% (131/131), 1.13 MiB | 10.92 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [6]:
# Установка необходимых библиотек
!pip install click requests tqdm pyspng ninja imageio-ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.9/205.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 9.4 MB/s eta 0:00:00


In [7]:
# Импортируем нужные модули
import os
import random
import shutil
from zipfile import ZipFile
from google.colab import drive
from PIL import Image

# Монтирование Google Диска
drive.mount('/content/drive')

# Путь к zip файлу
zip_file_path = "/content/drive/MyDrive/archive.zip"

# Создаем директорию для распаковки архива
os.makedirs("anime_gan", exist_ok=True)

# Распаковка архива
with ZipFile(zip_file_path, "r") as zipobj:
    zipobj.extractall("anime_gan")

# Путь к вашему исходному датасету
source_dir = "anime_gan/images"
# Путь к новому датасету
target_dir = "anime_gan/small_dataset"
class_dir = os.path.join(target_dir, "class1")

# Создаем целевую директорию и фиктивную директорию класса, если они не существуют
os.makedirs(class_dir, exist_ok=True)

# Получаем список всех файлов в исходной директории
all_files = os.listdir(source_dir)

# Перемешиваем файлы и выбираем первые 5000
random.shuffle(all_files)
selected_files = all_files[:5000]

# Копируем выбранные файлы в новую директорию класса
for file_name in selected_files:
    source_file = os.path.join(source_dir, file_name)
    target_file = os.path.join(class_dir, file_name)
    shutil.copy(source_file, target_file)

print(f"Скопировано {len(selected_files)} файлов в {class_dir}")

# Ресайз изображений до 64x64
for file_name in os.listdir(class_dir):
    img_path = os.path.join(class_dir, file_name)
    image = Image.open(img_path)
    image = image.resize((64, 64))
    image.save(img_path)


Mounted at /content/drive
Скопировано 5000 файлов в anime_gan/small_dataset/class1


In [7]:
# Удалить папку и все ее содержимое
!rm -r /content/anime_gan/small_dataset

In [8]:
# Упаковываем изображения в архив
!zip -r /content/anime_gan/small_dataset.zip /content/anime_gan/small_dataset

# Подготавливаем данные для StyleGAN2-ADA
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source=/content/anime_gan/small_dataset --dest=/content/datasets/anime_gan.zip

# Запускаем обучение
!python /content/stylegan2-ada-pytorch/train.py --outdir=/content/training-runs --data=/content/datasets/anime_gan.zip --gpus=1

Выходные данные были обрезаны до нескольких последних строк (5000).
  adding: content/anime_gan/small_dataset/class1/40639_2013.jpg (deflated 7%)
  adding: content/anime_gan/small_dataset/class1/21692_2008.jpg (deflated 7%)
  adding: content/anime_gan/small_dataset/class1/47811_2014.jpg (deflated 7%)
  adding: content/anime_gan/small_dataset/class1/62065_2019.jpg (deflated 7%)
  adding: content/anime_gan/small_dataset/class1/10884_2005.jpg (deflated 7%)
  adding: content/anime_gan/small_dataset/class1/468_2000.jpg (deflated 8%)
  adding: content/anime_gan/small_dataset/class1/48506_2014.jpg (deflated 7%)
  adding: content/anime_gan/small_dataset/class1/55488_2016.jpg (deflated 6%)
  adding: content/anime_gan/small_dataset/class1/58149_2017.jpg (deflated 7%)
  adding: content/anime_gan/small_dataset/class1/42633_2013.jpg (deflated 6%)
  adding: content/anime_gan/small_dataset/class1/26211_2009.jpg (deflated 7%)
  adding: content/anime_gan/small_dataset/class1/12489_2005.jpg (deflated 6%